In [1]:
!pip install datasets

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached xxhash-3.5.0-cp312-cp312-win_amd64.whl (30 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplerllm 0.3.1.1 requires Requests==2.31.0, but you have requests 2.32.3 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from datasets import load_dataset

def download_parquet_dataset(dataset_name: str, save_path: str, split: str = "train"):
    """
    Downloads a dataset from Hugging Face stored in Parquet format and saves it locally.
    
    Args:
        dataset_name (str): The name of the dataset on Hugging Face.
        save_path (str): The local directory path where the dataset should be saved.
        split (str): The dataset split to download (default: "train").
        
    Returns:
        None
    """
    try:
        # Load the dataset
        print(f"Downloading dataset '{dataset_name}' ({split} split)...")
        dataset = load_dataset(dataset_name, split=split)
        
        # Save the dataset to the specified path
        save_file = f"{save_path}/{dataset_name}_{split}.parquet"
        # dataset.to_parquet(save_file)
        return dataset
        
        # print(f"Dataset saved at {save_file}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
if __name__ == "__main__":
    # Replace 'your-dataset-name' with the dataset ID from Hugging Face
    # Replace 'your/local/path' with the directory to save the dataset
    download_parquet_dataset("huggingface_dataset_id", "your/local/path")


e:\Prajwal\Advance MCQ Generator\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


An error occurred: Dataset 'huggingface_dataset_id' doesn't exist on the Hub or cannot be accessed.


In [54]:
open_ended = download_parquet_dataset(dataset_name="rajpurkar/squad", save_path="data/datasets/open_ended.parquet", split="validation")

In [56]:
open_ended

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [71]:
mcq = download_parquet_dataset(dataset_name="allenai/qasc", save_path="data/datasets/mcq.parquet", split="test")

In [15]:
mcq

Dataset({
    features: ['id', 'question', 'choices', 'answerKey', 'fact1', 'fact2', 'combinedfact', 'formatted_question'],
    num_rows: 8134
})

In [ ]:
input_open = # context

In [ ]:
output_open = """
question: {question} \n
answer: {answer}
""" 

In [21]:
open_ended

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [59]:
from datasets import load_dataset, Dataset

def convert_to_instruction_format_open(dataset: str, instruction: str, split: str = "train"):
    """
    Converts a Hugging Face dataset to a new format with features: ['instruction', 'input', 'output'].

    Args:
        dataset_name (str): The name of the dataset on Hugging Face.
        instruction (str): The instruction text to include for all rows.
        split (str): The dataset split to load and transform (default: "train").

    Returns:
        new_dataset: A transformed dataset in the new format.
    """
    # Load the dataset
    
    # Define the conversion function
    def transform_row(row):
        input_text = f"Title: {row['title']}\nContext: {row['context']}"
        output_text =f"""
question: {row["question"]} \n
answer: {(row["answers"]["text"][0])}
""" 
        return {
            "instruction": instruction,
            "input": input_text,
            "output": output_text,
            "id": row['id']
        }
    
    # Apply the transformation
    print("Converting dataset...")
    new_data = dataset.map(transform_row, remove_columns=['id', 'title', 'context', 'question', 'answers'])
    
    return new_data




In [60]:
# Example usage
template_ope= """Given the following context create me open ended question-answer pair"""

new_dataset = convert_to_instruction_format_open(open_ended, instruction=template_ope)

Converting dataset...


Map: 100%|██████████| 10570/10570 [00:00<00:00, 16813.45 examples/s]


In [63]:
new_dataset

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 10570
})

In [61]:
import itertools
m = 5
print("Instruction-tuned dataset:")
top_m = list(itertools.islice(new_dataset, m))
for j in top_m:
  print(j)

Instruction-tuned dataset:
{'id': '56be4db0acb8001400a502ec', 'instruction': 'Given the following context create me open ended question-answer pair', 'input': 'Title: Super_Bowl_50\nContext: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', 'output': '\nquestion: Which N

In [64]:
new_dataset.to_parquet("./dataset/val_open.parquet")

Creating parquet from Arrow format: 100%|██████████| 11/11 [00:00<00:00, 845.84ba/s]


10815959

In [65]:
mcq

Dataset({
    features: ['id', 'question', 'choices', 'answerKey', 'fact1', 'fact2', 'combinedfact', 'formatted_question'],
    num_rows: 926
})

In [43]:
template_mcq= """Given the following context create me MCQ questions with options and answer pair"""



In [45]:
from datasets import load_dataset, Dataset

def convert_to_instruction_format_mcq(dataset: str, instruction: str, split: str = "train"):
    """
    Converts a Hugging Face dataset to a new format with features: ['instruction', 'input', 'output'].

    Args:
        dataset_name (str): The name of the dataset on Hugging Face.
        instruction (str): The instruction text to include for all rows.
        split (str): The dataset split to load and transform (default: "train").

    Returns:
        new_dataset: A transformed dataset in the new format.
    """
    # Load the dataset
    
    # Define the conversion function
    def transform_row(row):
        input_text = row['combinedfact']
        output_text =f"""
question: {row["formatted_question"]} \n
answer: {(row["answerKey"])}
""" 
        return {
            "instruction": instruction,
            "input": input_text,
            "output": output_text,
            "id": row['id']
        }
    
    # Apply the transformation
    print("Converting dataset...")
    new_data = dataset.map(transform_row, remove_columns=['question', 'choices', 'answerKey', 'fact1', 'fact2', 'combinedfact', 'formatted_question'])
    
    return new_data

In [72]:
mcq_updated = convert_to_instruction_format_mcq(dataset=mcq, instruction=template_mcq)

Converting dataset...


Map: 100%|██████████| 920/920 [00:00<00:00, 17795.67 examples/s]


In [73]:
mcq_updated

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 920
})

In [74]:
new_dataset

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 10570
})

In [75]:
import itertools
m = 5
print("Instruction-tuned dataset:")
top_m = list(itertools.islice(mcq_updated, m))
for j in top_m:
  print(j)

Instruction-tuned dataset:
{'id': '3C44YUNSI1OBFBB8D36GODNOZN9DPA', 'instruction': 'Given the following context create me MCQ questions with options and answer pair', 'input': '', 'output': '\nquestion: What type of birth do therian mammals have? (A) Anthax (B) under water (C) uterus (D) wombs (E) two (F) moles (G) live (H) embryo \n\nanswer: \n'}
{'id': '3B1NLC6UGZVERVLZFT7OUYQLD1SGPZ', 'instruction': 'Given the following context create me MCQ questions with options and answer pair', 'input': '', 'output': '\nquestion: By what time had mouse-sized viviparous mammals evolved? (A) Corvidae (B) arthropods (C) birds (D) backbones (E) keratin (F) Jurassic (G) front paws (H) Parakeets. \n\nanswer: \n'}
{'id': '3QRYMNZ7FYGITFVSJET3PS0F4S0NT9', 'instruction': 'Given the following context create me MCQ questions with options and answer pair', 'input': '', 'output': "\nquestion: What does a plant's skin do? (A) Reduced friction (B) causes infection (C) vital to a good life (D) prevents water lo

In [76]:
mcq_updated.to_parquet("./dataset/test_mcq.parquet")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 499.86ba/s]


287267

In [82]:
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset
import os

def concatenate_parquet_with_hf(parquet_files: str, output_file: str = None):
    """
    Reads multiple Parquet files as Hugging Face datasets and concatenates them vertically.

    Args:
        input_folder (str): The folder containing the Parquet files.
        output_file (str): Optional. The path to save the concatenated dataset.

    Returns:
        Dataset: The concatenated Hugging Face dataset.
    """
    if not parquet_files:
        raise ValueError("No Parquet files found in the specified folder.")
    
    print(f"Found {len(parquet_files)} files. Reading and concatenating...")

    # Load each Parquet file as a Hugging Face dataset
    datasets = [load_dataset("parquet", data_files=file)["train"] for file in parquet_files]
    
    # Concatenate the datasets
    concatenated_dataset = concatenate_datasets(datasets)
    
    # Optionally save the concatenated dataset
    if output_file:
        concatenated_dataset.to_parquet(output_file)
        print(f"Concatenated dataset saved to {output_file}")
    
    return concatenated_dataset


In [84]:
train_list = [r'dataset/train_mcq.parquet', r'dataset/train_open.parquet']

train_merge = concatenate_parquet_with_hf(train_list, "./dataset/train_merge.parquet")

Found 2 files. Reading and concatenating...


Creating parquet from Arrow format: 100%|██████████| 96/96 [00:00<00:00, 732.84ba/s]

Concatenated dataset saved to ./dataset/train_merge.parquet


In [85]:
test_list = [r'dataset/test_mcq.parquet', r'dataset/val_mcq.parquet', r'dataset/val_open.parquet']

train_merge = concatenate_parquet_with_hf(test_list, "./dataset/test_merge.parquet")

Found 3 files. Reading and concatenating...


Creating parquet from Arrow format: 100%|██████████| 13/13 [00:00<00:00, 721.12ba/s]

Concatenated dataset saved to ./dataset/test_merge.parquet
